In [1]:
import pandas as pd
import csv
import json
import requests
import os
import pymongo
from pymongo import MongoClient
from sqlalchemy import create_engine
from pandas.io.json import json_normalize

In [2]:
hci_low_income = pd.read_csv("Data/hci_low_income.csv")
hci_low_income = hci_low_income[["race_eth_name","geoname","county_name","fam_lt_lw","families","livingwage"]]
hci_low_income_clean = hci_low_income.dropna(how="any", axis = 0)
hci_low_income_clean = hci_low_income_clean.rename(columns={"race_eth_name": "race", "geoname": "geoname","county_name": "county", "fam_lt_lw":"families_below_the_living_wage", "families": "number_of_families", "livingwage": "living_wage"})
hci_low_income_clean.head()

,race,geoname,county,families_below_the_living_wage,number_of_families,living_wage
9,AIAN,Alameda,Alameda,71.0,320.0,22.64
10,Asian,Alameda,Alameda,5735.0,41863.0,22.64
11,AfricanAm,Alameda,Alameda,1149.0,6431.0,22.64
12,Latino,Alameda,Alameda,9281.0,25776.0,22.64
13,NHOPI,Alameda,Alameda,118.0,1046.0,22.64


In [3]:
hci_low_income_clean_grouped = hci_low_income_clean.groupby(["county"]).mean().reset_index()
hci_low_income_clean_grouped["percent_of_families_below_the_living_wage"] = hci_low_income_clean_grouped["families_below_the_living_wage"]/hci_low_income_clean_grouped["number_of_families"] *100
hci_low_income_clean_grouped = hci_low_income_clean_grouped.sort_values("county", ascending = True)
hci_low_income_clean_grouped = hci_low_income_clean_grouped.reset_index()
del hci_low_income_clean_grouped['index']
hci_low_income_clean_grouped = hci_low_income_clean_grouped.round({'families_below_the_living_wage': 2, 'number_of_families': 2, 'living_wage':2, 'percent_of_families_below_the_living_wage':2})
hci_low_income_clean_grouped.head()

,county,families_below_the_living_wage,number_of_families,living_wage,percent_of_families_below_the_living_wage
0,Alameda,2400.72,8062.14,24.73,29.78
1,Alpine,2.75,34.50,21.36,7.97
2,Amador,126.50,403.28,22.44,31.37
3,Butte,549.40,1373.60,20.88,40.00
4,Calaveras,90.00,390.12,20.87,23.07


In [4]:
hci_low_income_clean_grouped.to_csv("Data/cleaned_lowincome.csv", encoding="utf-8", index=False, header=True)

In [5]:
hci_low_income_clean_race = hci_low_income_clean[["race","families_below_the_living_wage","number_of_families"]]
hci_low_income_clean_race = hci_low_income_clean_race.groupby(["race"]).sum()
hci_low_income_clean_race["percent_of_families_below_the_living_wage"] = round(hci_low_income_clean_race["families_below_the_living_wage"]/
                            hci_low_income_clean_race["number_of_families"]*100, 2)
hci_low_income_clean_race = hci_low_income_clean_race.reset_index()
hci_low_income_clean_race = hci_low_income_clean_race.sort_values("percent_of_families_below_the_living_wage", ascending = False)
hci_low_income_clean_race = hci_low_income_clean_race.drop([7])
hci_low_income_clean_race

,race,families_below_the_living_wage,number_of_families,percent_of_families_below_the_living_wage
1,AfricanAm,120611.0,223673.0,53.92
3,Latino,823435.0,1563398.0,52.67
0,AIAN,6447.0,14260.0,45.21
6,Other,3991.0,10341.0,38.59
4,Multiple,20147.0,59334.0,33.96
5,NHOPI,4318.0,13534.0,31.90
2,Asian,125003.0,516633.0,24.20
8,White,304113.0,1469036.0,20.70


In [6]:
hci_low_income_clean_race.to_csv("Data/cleaned_lowincomegrouped.csv", encoding="utf-8", index=False, header=True)

In [7]:
#Load the dataframes to Mongo DB using a new function

def write_df_to_mongoDB(  my_df,\
                          database_name = 'mydatabasename' ,\
                          collection_name = 'mycollectionname',
                          server = 'localhost',\
                          mongodb_port = 27017,\
                          chunk_size = 100):

    client = MongoClient('localhost',int(mongodb_port))
    db = client[database_name]
    collection = db[collection_name]
    # To write
    collection.delete_many({})  # Destroy the collection
    #aux_df=aux_df.drop_duplicates(subset=None, keep='last') # To avoid repetitions
    my_list = my_df.to_dict('records')
    l =  len(my_list)
    ran = range(l)
    steps=list(ran[chunk_size::chunk_size])
    steps.append([l])

    # Insert chunks of the dataframe
    i = 0
    for j in steps:
        if type(j) != list:
            collection.insert_many(my_list[i:j]) 
        i = j

    print('Done')
    return

In [8]:
medically_underserved_data = json.load(open('Data/Medically_Underserved_Areas.geojson'))
medically_underserved_df = pd.DataFrame(medically_underserved_data["features"])
medically_underserved_df.head()

,type,properties,geometry
0,Feature,"{'OBJECTID': 1, 'OBJECTID_1': 198, 'MuaSrcID':...","{'type': 'Polygon', 'coordinates': [[[-121.495..."
1,Feature,"{'OBJECTID': 2, 'OBJECTID_1': 199, 'MuaSrcID':...","{'type': 'Polygon', 'coordinates': [[[-121.571..."
2,Feature,"{'OBJECTID': 3, 'OBJECTID_1': 200, 'MuaSrcID':...","{'type': 'Polygon', 'coordinates': [[[-121.621..."
3,Feature,"{'OBJECTID': 4, 'OBJECTID_1': 201, 'MuaSrcID':...","{'type': 'Polygon', 'coordinates': [[[-124.316..."
4,Feature,"{'OBJECTID': 5, 'OBJECTID_1': 202, 'MuaSrcID':...","{'type': 'Polygon', 'coordinates': [[[-119.731..."


In [9]:
with open('Data/medical_county.json') as json_data:
    data = json.load(json_data)
medical_county = pd.DataFrame(data['records'])
medical_county = pd.DataFrame(medical_county.rename(columns={0: 'County',1: 'Provider',2: 'Address',3: 'City',
                4: 'State',5: 'Zip',6: 'Phone',7: 'Website',8: 'Category',9: 'Type',10: 'Services',
                11: 'Latitute',12: 'Longitude',13: 'Location'}))
medical_county.head()

,County,Provider,Address,City,State,Zip,Phone,Website,Category,Type,Services,Latitute,Longitude,Location
0,Alameda,Oakland Community Action Partnership,"150 Frank H. Ogawa Plaza 4th Floor, Suite 4340",Oakland,CA,94612,(510) 238-2362,www.AC-OCAP.com,Community Services,Community Services,Child/Youth Services,37.805293900000002,-122.2711085,"(37.805293900000002, -122.2711085)"
1,Alameda,Oakland Community Action Partnership,"150 Frank H. Ogawa Plaza 4th Floor, Suite 4340",Oakland,CA,94612,(510) 238-2362,www.AC-OCAP.com,Community Services,Community Services,Education Services,37.805293900000002,-122.2711085,"(37.805293900000002, -122.2711085)"
2,Alameda,Oakland Community Action Partnership,"150 Frank H. Ogawa Plaza 4th Floor, Suite 4340",Oakland,CA,94612,(510) 238-2362,www.AC-OCAP.com,Community Services,Community Services,Emergency Services,37.805293900000002,-122.2711085,"(37.805293900000002, -122.2711085)"
3,Alameda,Oakland Community Action Partnership,"150 Frank H. Ogawa Plaza 4th Floor, Suite 4340",Oakland,CA,94612,(510) 238-2362,www.AC-OCAP.com,Community Services,Community Services,Employment Services,37.805293900000002,-122.2711085,"(37.805293900000002, -122.2711085)"
4,Alameda,Oakland Community Action Partnership,"150 Frank H. Ogawa Plaza 4th Floor, Suite 4340",Oakland,CA,94612,(510) 238-2362,www.AC-OCAP.com,Community Services,Community Services,Health Services,37.805293900000002,-122.2711085,"(37.805293900000002, -122.2711085)"


In [10]:
#Use above function to create a collection to MongoDB
write_df_to_mongoDB(medical_county,\
                          database_name = 'HealthcareProject' ,\
                          collection_name = 'medical_county',
                          server = 'localhost',\
                          mongodb_port = 27017,\
                          chunk_size = 10) 

Done


In [11]:
write_df_to_mongoDB(medically_underserved_df,\
                          database_name = 'HealthcareProject' ,\
                          collection_name = 'medically_underserved_df',
                          server = 'localhost',\
                          mongodb_port = 27017,\
                          chunk_size = 100)

Done


In [12]:
# Upload CSV files to SQL
rds_connection_string = "admin2:12345@localhost:5432/Healthcare_LowIncome"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [ ]:
## to create the user

# CREATE USER admin22 with Password '12345'
# Alter User admin22 With SuperUser;


In [ ]:
## to remove if database exists

# IF EXISTS(SELECT *
#           FROM   dbo.Healthcare_LowIncome)
#   DROP TABLE dbo.low_income
#   DROP TABLE dbo.low_income_race

In [ ]:
## to create the two data

# CREATE TABLE low_income(
# 	County VARCHAR PRIMARY KEY,
# 	Families_Below_the_Living_Wage INT,
# 	Number_of_Families INT,
# 	Living_Wage INT,
# 	Percent_of_Families_Below_the_Living_Wage INT
# );

In [ ]:
# CREATE TABLE low_income_race(
# 	Race VARCHAR,
# 	County VARCHAR,
# 	Families_Below_the_Living_Wage INT,
# 	Number_of_Families INT,
# 	Living_Wage INT,
# 	Percent_of_Families_Below_the_Living_Wage INT,
# 	FOREIGN KEY (County) REFERENCES low_income(County)
# );

In [13]:
# make sure the tables do not already exist in database Healthcare_LowIncome
engine.table_names()

['low_income', 'low_income_race']

In [14]:
hci_low_income_clean_grouped.to_sql(name='low_income', con=engine, if_exists='append', index=False)

In [15]:
hci_low_income_clean_race.to_sql(name='low_income_race', con=engine, if_exists='append', index=False)

In [16]:
pd.read_sql_query('select * from low_income', con=engine).head()

,county,families_below_the_living_wage,number_of_families,living_wage,percent_of_families_below_the_living_wage
0,Alameda,2401,8062,25,30
1,Alpine,3,35,21,8
2,Amador,127,403,22,31
3,Butte,549,1374,21,40
4,Calaveras,90,390,21,23


In [17]:
pd.read_sql_query('select * from low_income_race', con=engine).head()

,race,county,families_below_the_living_wage,number_of_families,living_wage,percent_of_families_below_the_living_wage
0,AfricanAm,None,120611,223673,None,54
1,Latino,None,823435,1563398,None,53
2,AIAN,None,6447,14260,None,45
3,Other,None,3991,10341,None,39
4,Multiple,None,20147,59334,None,34


In [18]:
engine.table_names()

['low_income', 'low_income_race']